In [1]:
#r "./src/Model/bin/Debug/net6.0/Model.dll"

open Model

The below script needs to be able to find the current output cell; this is an easy method to get it.

# What is this?

* Have you ever felt the problem with architectural diagrams is they are _too visual_?
* Have you ever thought, "this diagram is great in theory, but what if I could validate it somehow?"
* Have you thought that your diagrams do too little, and by focussing on one aspect of your architecture they are too clear?

## Well, this is the solution

I was looking for a way to visualise a complex architecture, that allowed me to include some often-overlooked concepts as _reliability_ and _ownership_. Not finding a modelling tool that would fit my needs, I turned to what I always turned to - I wrote some code.

This is the result, a simple DSL for describing architectures that can then be used to generate diagrams of a variety of flavours, from a single source of truth.

### No, really, Why?

By creating a DSL in code, we gain two things.

1. We can create complex hierarchies that are difficult to visualise (but still reflective of the truth of the system), and then create simplified visualisations without losing fidelity.
1. We can test our architecture, validate best practices and search for pain points directly from the architecure model.

By modelling the architecture, and not simply diagramming it, we can test our assumptions, validate our behaviours and estimate the reliability and performance of your architecture before you've written a single line of code. (Except for these ones.)

## Enough waffle

You're right, let's do a demonstration.

Imagine you have an architecture, consisting of a single application with no dependencies, that was written by a mythical perfect engineer, and has no bugs. You might model that like so:

In [4]:
let ``my perfect application`` = {
        name = "a totally reliable service"
        links = [||]
        serviceType = InternalService
        reliabilityProfile = randomUptimeProfile 1.0
        metadata = None
    }

You can now validate that your perfect application is actually perfect. Using the method `walkService` which runs a test operation against the application.

In [5]:
#r "nuget: Xunit, *-*"

open Xunit

[<Fact>]
let ``Reliable services are always reliable`` () =
    let result = walkService ``my perfect application`` 

    Assert.StrictEqual(ServiceLevel.Unavailable, result)

// Run the test, we're in a notebook, not a test runner

``Reliable services are always reliable``()

Installed Packages Xunit, 2.4.2

Unhandled exception: Xunit.Sdk.EqualException: Assert.Equal() Failure
Expected: Unavailable
Actual:   Working
   at Xunit.Assert.Equal[T](T expected, T actual, IEqualityComparer`1 comparer) in /_/src/xunit.assert/Asserts/EqualityAsserts.cs:line 100
   at Xunit.Assert.StrictEqual[T](T expected, T actual) in /_/src/xunit.assert/Asserts/EqualityAsserts.cs:line 238
   at <StartupCode$FSI_0009>.$FSI_0009.main@()

Oh wait, our test was wrong. Perfect services are always working, not unavailable.

But, you get the picture.

What if our perfect service depends on another service? 

Here we're using the `randomUptimeProfile` function that models a service with a given chance of success.

In [8]:
let ``Our unreliable dependency`` = {
    name = "a totally unreliable service"
    links = [||]
    serviceType = InternalService
    reliabilityProfile = randomUptimeProfile 0.5
    metadata = None
}

let ``my perfect application`` = {
        name = "a totally reliable service"
        links = [|Requires(``Our unreliable dependency``)|] |> noMetadata
        serviceType = InternalService
        reliabilityProfile = randomUptimeProfile 1.0
        metadata = None
    }

// This should work fine, right?
display(walkService ``my perfect application``)
display(walkService ``my perfect application``)
display(walkService ``my perfect application``)

Working

Working

Working

Oh yes, it's not quite so reliable anymore. But, we don't just need to run `walkService` a bunch of times. We can get the testing framework to do that for us. The `determineServiceUptime` runs `walkService` a bunch of times and aggregates the success, failures, and degradations (we'll get to that later).

In [9]:
let success, failures, _ = determineServiceUptime 1000 ``my perfect application`` 

sprintf "successes: %d, failures: %d\n" success failures

successes: 512, failures: 488


Oh yes, that makes sense. What would we do in our architecture to improve the reliability? Implement retries with exponential backoff! 

By feeding the `randomUptimeProfile` through the `retrying` strategy we can model a system with retries.

In [11]:
open Reliability.Patterns

let ``Our unreliable dependency`` = {
    name = "a totally unreliable service"
    links = [||]
    serviceType = InternalService
    reliabilityProfile = randomUptimeProfile 0.5
    metadata = None
}

let ``my nearly perfect application`` = {
        name = "a totally reliable service"
        links = [|Requires(``Our unreliable dependency`` |> mitigatedBy (retrying 3))|] |> noMetadata
        serviceType = InternalService
        reliabilityProfile = randomUptimeProfile 1.0
        metadata = None
    }

let success, failures, _ = determineServiceUptime 1000 ``my nearly perfect application`` 

sprintf "successes: %d, failures: %d\n" success failures

successes: 1000, failures: 0


Oh yes, that's much better.